# Выбор локации для скважины

**Описание проекта**

Цель нашего проекта  - для нефтедобывающей компании определить регион с наибольшей прибылью и минимальными рисками убытков из трех потенциальных к разработке.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Возможную прибыль и риски необходимо пранализировать техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Описание данных**
 - `id` — уникальный идентификатор скважины;
 - `f0`, `f1`, `f2` — три значимых признака точек;
 - `product` — объём запасов в скважине (тыс. баррелей).

**Условия задачи**
 - Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
 - При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
 - Бюджет на разработку скважин в регионе — 10 млрд рублей.
 - При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
 - После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

### Импорт и просмотр данных

Импортируем необходимые библиотеки и загружаем данные

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# цвета для форматирования текста
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [2]:
data_0 = pd.read_csv('https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1j5jfpTN_sqVq2_xEtSk8AKJ7DQcqjEID')
data_1 = pd.read_csv('https://drive.google.com/uc?export=download&confirm=no_antivirus&id=17XHmdTED5tHexHhFifR37KxnaoMGVkgB')
data_2 = pd.read_csv('https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1jdHrX77e68JQwBob4UEZpn8jrE40r0bj')

Используем функцию для первичного просмотра данных

In [3]:
def data_review(data):
    display(data.sample(7))
    print('-'*100)
    data.info()
    print('-'*100)
    print('Пропуски в данных')
    display(data.isna().mean().sort_values(ascending=False))
    print('-'*100)
    print(f'Количество явных дубликатов - {data.duplicated().sum()}')
    print('-'*100)
    display(data.describe())

In [4]:
data_review(data_0)

,id,f0,f1,f2,product
62477,KstPC,0.315324,1.024940,4.167427,72.738596
32857,JM58J,-1.114045,0.136319,-3.722147,34.357856
47322,WVGzj,1.760809,0.193551,-1.706838,47.063902
69812,ZZPPg,1.923356,-0.074721,1.414870,155.715535
84036,3BwZf,0.884271,0.530905,-0.113340,39.182012
21376,Dlbwk,-0.091263,0.440606,1.496223,135.865039
90628,qbu1Y,1.838208,0.214371,1.197145,47.012629


----------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
----------------------------------------------------------------------------------------------------
Пропуски в данных


id         0.0
f0         0.0
f1         0.0
f2         0.0
product    0.0
dtype: float64

----------------------------------------------------------------------------------------------------
Количество явных дубликатов - 0
----------------------------------------------------------------------------------------------------


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [8]:
data_review(data_1)

,id,f0,f1,f2,product
23879,3caYe,1.481587,2.391841,3.992239,107.813044
32962,Kf64G,-9.064941,-6.616521,3.000042,84.038886
90075,YncX4,11.852045,2.312248,3.996478,107.813044
66726,1Tdtw,-15.425620,-6.836418,-0.000819,3.179103
33914,hjMiE,-6.652612,-5.950137,2.001342,57.085625
42555,DkcbI,-3.335292,-15.694208,2.990927,84.038886
28382,Y8o4V,5.874583,2.269539,1.009366,26.953261


----------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
----------------------------------------------------------------------------------------------------
Пропуски в данных


id         0.0
f0         0.0
f1         0.0
f2         0.0
product    0.0
dtype: float64

----------------------------------------------------------------------------------------------------
Количество явных дубликатов - 0
----------------------------------------------------------------------------------------------------


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [5]:
data_review(data_2)

,id,f0,f1,f2,product
44487,hq381,-2.408979,-1.358955,2.228002,161.813034
65358,abLYb,0.208298,-1.172395,5.180625,63.434536
78494,m2f3p,0.640455,-0.116639,2.398754,109.598429
93483,WgHBt,-0.378945,2.161715,-2.148717,66.174435
49513,yuFwF,-0.569204,0.209268,2.681097,78.711301
3799,FaR8M,-3.945288,-2.190403,-0.004754,114.747195
97132,y00CL,-1.031678,0.357295,7.192593,107.000742


----------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
----------------------------------------------------------------------------------------------------
Пропуски в данных


id         0.0
f0         0.0
f1         0.0
f2         0.0
product    0.0
dtype: float64

----------------------------------------------------------------------------------------------------
Количество явных дубликатов - 0
----------------------------------------------------------------------------------------------------


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


**Вывод**

Данные геологоразведки всех трех регионов загрузились корректно. Пропусков и дубликатов в данных нет.  

Из анализа статистических показателей выделяются регионы `0` и `2` по среднему объём запасов в скважине (92.5 и 95 тыс. баррелей соответственно) относительно региона `1` (68.8 тыс. баррелей)

### Подготовка признаков

Для обучения модели в каждом регионе нас интересуют признаки `f0`, `f1`, `f2`, все они являются количественными и мы их отмасштабируем для нашей модели методом стандартизации данных.  
Целевым признаком является `product`.

На этапе подготовки данных для каждого региона:
* выделим в отдельные переменные признаки для обучения модели и целевой признак
* разделим данные на обучающую и валидационную выборки в соотношении 75:25
* отмасштабируем признаки для модели

In [6]:
# напишем функцию для подготовки данных для обучения модели
def data_preprocessing(data):
    x = data[['f0', 'f1', 'f2']].copy()
    y = data['product']
    x_train, x_val, y_train, y_val = train_test_split(x, 
                                                      y, 
                                                      test_size=0.25, 
                                                      random_state=42)
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_val = scaler.transform(x_val)
    return x_train, x_val, y_train, y_val        

In [7]:
# применим функцию к каждому региону
x0_train, x0_val, y0_train, y0_val = data_preprocessing(data_0)
x1_train, x1_val, y1_train, y1_val = data_preprocessing(data_1)
x2_train, x2_val, y2_train, y2_val = data_preprocessing(data_2)

**Вывод**

На этапе подготовки данных:
* выделены признаки для обучения и целевой признак 
* данные поделены на обучающую и валидационную выборки
* количественные признаки отмасштабированы

Все готово для обучения модели.

## Обучение и проверка модели

По нашим данным для каждого региона:
* обучим модель линейной регрессии 
* сделаем по модели предсказания запасов сырья 
* вычислим среднеквадратичную ошибку предсказаний

In [8]:
def model_learning(x_train, x_val, y_train, y_val, region):
    model = LinearRegression()
    model.fit(x_train, y_train)
    y_val = y_val.reset_index()   
    y_val['predict'] = model.predict(x_val)
    y_val = y_val.set_index(y_val['index'], drop=True).drop('index', axis=1)
    rmse = mean_squared_error(y_val['product'], y_val['predict'], squared=False)
    print(color.UNDERLINE + color.BOLD + f'Регион {region}' + color.END)
    print("Средний запас предсказанного сырья на скважину - " + 
          color.BOLD + f"{y_val['predict'].mean():.2f} тыс. баррелей" + color.END)
    print("RMSE модели для региона - " +
          color.BOLD + f"{rmse:.2f} тыс. баррелей" + color.END)
    print('---')
    return y_val

In [9]:
y0_val = model_learning(x0_train, x0_val, y0_train, y0_val, '0')
y1_val = model_learning(x1_train, x1_val, y1_train, y1_val, '1')
y2_val = model_learning(x2_train, x2_val, y2_train, y2_val, '2')

Регион 0
Средний запас предсказанного сырья на скважину - 92.40 тыс. баррелей
RMSE модели для региона - 37.76 тыс. баррелей
---
Регион 1
Средний запас предсказанного сырья на скважину - 68.71 тыс. баррелей
RMSE модели для региона - 0.89 тыс. баррелей
---
Регион 2
Средний запас предсказанного сырья на скважину - 94.77 тыс. баррелей
RMSE модели для региона - 40.15 тыс. баррелей
---


**Вывод**

На данном этапе мы для каждого региона обучили модель линейной регрессии и по ней сделали предсказания объемов запасов в скважинах по регионам.  

Можно отметить следующее:
* наиболее перспективными на данном этапе выглядят регионы `0` и `2` с бо'льшими средними запасами сырья на скважину относительно региона `1`
* но при этом стоит отметить, что для регионов `0` и `2` достаточно велика среднеквадратичная ошибка предсказаний, относительно средних значений запасов
* для региона `1` эта среднеквадратичное отклонение предсказаний крайне мало, что говорит нам о том, что модель в регионе `1` достаточно надежно и точно предсказывает запасы сырья, относительно двух других регионов

На этапе расчета прибыли и рисков нам будет необходимо определить стоит ли рисковать, направляя работы в регионы с большими запасами (`0` и `2`), но менее предсказуемыми со стороны модели, или остановиться на регионе с меньшими запасами, но более стабильном по предсказаниям модели (`1`).

## Подготовка к расчёту прибыли

Сохраним в переменных все ключевые экономические значения для расчета:
* **expense** - бюджет на разработку скважин в регионе - 10 млрд. рублей
* **income_per_1000barrel** - доход с тысячи баррелей нефти - 450 тыс. рублей
* **number_exploration_holes** - количество разведочных скважин в регионе - 500 скважин
* **number_operation_holes** - количество скважин для разработки в регионе - 200 скважин

In [10]:
EXPENCE = 10 * 10 ** 9
INCOME_PER_1000BARREL = 450 * 10**3
NUM_EXPLORATION_HOLES = 500
NUM_OPERATION_HOLES = 200

Рассчитаем минимальные значения запасов сырья в 200 наиболее перспективных скважинах региона для безубыточной разработки.

In [11]:
print (f'Минимальный запас сырья на месторождении - {(EXPENCE / INCOME_PER_1000BARREL / 1000):.1f} млн. баррелей')
print (f'Средний запас сырья на скважину - {(EXPENCE / (NUM_OPERATION_HOLES * INCOME_PER_1000BARREL)):.1f} тыс. баррелей')

Минимальный запас сырья на месторождении - 22.2 млн. баррелей
Средний запас сырья на скважину - 111.1 тыс. баррелей


**Вывод**
* для безубыточной разработки месторождения в регионе необходимо, чтобы 200 наиболее перспективных по модели скважин в сумме имели запасы не ниже 22.2 млн баррелей
* в среднем запас сырья должен быть не ниже 111.1 тыс. баррелей на скважину
* этот показатель превышает показатели средние запасы сырья на скважину в регионах 
* для безубыточной разработки необходимо, чтобы модель корректно выбирала топ-200 перспективных скважин в регионе

## Расчёт прибыли и рисков 

### Расчет прибыли

Напишем функцию для рассчета прибыль для каждого региона.

Функция будет:
* отбирать из нашей выборки 500 случайных точек (разведочные скважины)
* из этих 500 точек выбирать 200 наиболее перспективных по оценке модели (скважины в разработку)
* рассчитывать прибыль/убыток по результатам отработки запасов этих 200 скважин

In [16]:
def profit_calc(data, 
                region,
                r_state,
                sample_size=NUM_EXPLORATION_HOLES, 
                top_size=NUM_OPERATION_HOLES,
                print_res=False,
                replace=True):
    data_sample = data.sample(n=sample_size, random_state=r_state, replace=replace)
    top_predict = data_sample.sort_values(by='predict', ascending=False)[:top_size]
    reserve = top_predict['product'].sum()
    income = reserve * INCOME_PER_1000BARREL
    profit = income - EXPENCE
    if print_res:
        print(color.UNDERLINE + color.BOLD + f'Регион {region}' + color.END)
        print("Объем запасов в регионе - " + color.BOLD + 
              "{:,.1f}".format(reserve / 10**3).replace(',' , ' ') + " млн. баррелей" + color.END)
        if profit >= 0:
            print(color.GREEN + "Доход - " + color.BOLD +
                  "{:,.1f}".format(income / 10**6).replace(',' , ' ') + " млн. рублей" + color.END)
            print(color.GREEN + "Прибыль - " + color.BOLD + 
                  "{:,.1f}".format(profit / 10**6).replace(',' , ' ') + " млн. рублей" + color.END)
        else:
            print(color.RED + "Доход - " + color.BOLD + 
                  "{:,.1f}".format(income / 10**6).replace(',' , ' ') + " млн. рублей" + color.END)
            print(color.RED + "Убыток - " + color.BOLD + 
                  "{:,.1f}".format(profit / 10**6).replace(',' , ' ') + " млн. рублей" + color.END)
    else:
        return profit

Для начала посмотрим на максимально возможные результаты по регионам. Для этого смоделируем ситуацию, что мы из всей валидационной выборки (25000 значений) выбираем 200 наиболее перспективных.

In [17]:
profit_calc(y0_val, '0', replace=False, print_res=True, r_state=5678, sample_size=len(y0_val))
profit_calc(y1_val, '1', replace=False, print_res=True, r_state=5678, sample_size=len(y0_val))
profit_calc(y2_val, '2', replace=False, print_res=True, r_state=5678, sample_size=len(y0_val))

Регион 0
Объем запасов в регионе - 29.7 млн. баррелей
Доход - 13 359.1 млн. рублей
Прибыль - 3 359.1 млн. рублей
Регион 1
Объем запасов в регионе - 27.6 млн. баррелей
Доход - 12 415.1 млн. рублей
Прибыль - 2 415.1 млн. рублей
Регион 2
Объем запасов в регионе - 28.0 млн. баррелей
Доход - 12 598.6 млн. рублей
Прибыль - 2 598.6 млн. рублей


Наиболее эффективным в этом случае оказался регион 0, несколько отстают от него регионы 1 и 2. Все регионы при данном раскладе экономически эффективны.

Но такая ситуация практически исключена в реальной жизни. По условиям задачи у нас есть 500 скважин в регионе для выбора 200 наиболее перспективных из них по модели.

Посмотрим экономические результаты на случайной выборке из 500 разведочных скважин в каждом регионе.

In [18]:
profit_calc(y0_val, '0', replace=False, print_res=True, r_state=42)
profit_calc(y1_val, '1', replace=False, print_res=True, r_state=42)
profit_calc(y2_val, '2', replace=False, print_res=True, r_state=42)

Регион 0
Объем запасов в регионе - 22.6 млн. баррелей
Доход - 10 183.1 млн. рублей
Прибыль - 183.1 млн. рублей
Регион 1
Объем запасов в регионе - 23.2 млн. баррелей
Доход - 10 436.3 млн. рублей
Прибыль - 436.3 млн. рублей
Регион 2
Объем запасов в регионе - 23.3 млн. баррелей
Доход - 10 469.4 млн. рублей
Прибыль - 469.4 млн. рублей


Все регионы также показали прибыль на случайной выборке, причем прибыль уже во всех регионах близка к грани рентабельности. 
В лидерах регион 2.

### Расчет рисков

Для подсчета рисков получить убытки в регионе мы применим технику Bootstarp:
* сделаем 1000 случайных выборок 500 разведочных скважин 
* из которых выберем 200 перспективных по модели 
* рассчитаем прибыль/убыток при разработке этих 200 скважин
* рассчитаем среднюю прибыль
* рассчитем 95% доверительный интервал
* рассчитаем долю риска получить убытки при разработке месторождения в регионе

In [20]:
def bootstrap(data, 
              region,
              sample_size=NUM_EXPLORATION_HOLES, 
              top_size=NUM_OPERATION_HOLES,
              n_samples=1000):  
    profits = []
    state = np.random.RandomState(42)
    for _ in range(n_samples):
        profits.append(profit_calc(data, region, state, print_res=False))
    profits = pd.Series(profits) 
    print(color.UNDERLINE + color.BOLD + f'Регион {region}' + color.END)
    print('Средняя прибыль в регионе - ' + color.BOLD + 
          "{:,.1f}".format(profits.mean() / 10**6) + " млн. рублей" + color.END)
    print('95% доверительный интервал - от ' + color.BOLD + 
          '{:.1f}'.format(profits.quantile(0.025) / 10**6) + " млн. рублей" + color.END + " до " +
          color.BOLD + '{:.1f}'.format(profits.quantile(0.975) / 10**6) + " млн. рублей" + color.END)
    print('Доля убыточных вариантов (риск убытка) - ' + color.BOLD + 
          '{:.1%}'.format((profits < 0).mean()) + color.END)

In [22]:
bootstrap(y0_val, '0')
bootstrap(y1_val, '1')
bootstrap(y2_val, '2')

Регион 0
Средняя прибыль в регионе - 399.6 млн. рублей
95% доверительный интервал - от -110.5 млн. рублей до 897.5 млн. рублей
Доля убыточных вариантов (риск убытка) - 6.0%
Регион 1
Средняя прибыль в регионе - 452.0 млн. рублей
95% доверительный интервал - от 61.7 млн. рублей до 845.3 млн. рублей
Доля убыточных вариантов (риск убытка) - 1.5%
Регион 2
Средняя прибыль в регионе - 375.0 млн. рублей
95% доверительный интервал - от -144.8 млн. рублей до 888.4 млн. рублей
Доля убыточных вариантов (риск убытка) - 8.0%


При анализе риска убытков в каждом регионе только регион `1` показал вероятность убытков меньше, чем пороговое значение в условии - 2.5%. Риск убытков в регионе 1 - **1.5%**. Также в этом регионе максимальная средняя прибыль - 452 млн рублей.  
Регионы `0` и `2`, несмотря на их больший максимальный потенциал по ресурсам, являются более рискованными и в **6 и 8%** случаев соответственно месторождения в этих регионах окажутся убыточными.  
Таким образом для минимизации риска разработки нерентабельного проекта мы можем предложить направить работы в регион `1`, как самый стабильный и предсказуемый из трех.

## Общий вывод

**Целью нашего проекта** являлось для нефтедобывающей компании определение региона с наибольшей прибылью и минимальными рисками убытков из трех потенциальных к разработке.

По каждому региону нам были предоставлены пробы нефти по 10 000 точкам наблюдений (разведочным скважинам) для которых известны 3 параметра, от которых зависит объем запасов, и сообственно объем запасов сырья в скважине.

Нам было необходимо посторить модель линейной регрессии, которая поможет определить регион с наибольшим потенциалом прибыли.

**На начальном этапе** мы загрузили данные по трем регионам и ознакомились с ними. В данных пропусков и дубликатов нет. при первичном просмотре статистических показателей выделяются регионы `0` и `2` по среднему объём запасов в скважине (92.5 и 95 тыс. баррелей соответственно) относительно региона `1` (68.8 тыс. баррелей).

**Далее на этопе подготовки признаков** для обучения модели мы выделили в отдельные выборки признаки для обучения и целевой признак, затем поделили данные на обучающую и валидационную выборки и отмасштабировали количественные признаки.

Затем мы для каждого региона **обучили модели** линейной регрессии, сделали по ним предсказания и вычислили среднеквадратичное отклонение предсказаний от истинных значений. При анализе результатов мы отметили следующие вещи:
* наиболее перспективными на данном этапе выглядят регионы `0` и `2` с бо'льшими средними запасами сырья на скважину относительно региона `1`
* но при этом стоит отметить, что для регионов `0` и `2` достаточно велика среднеквадратичная ошибка предсказаний, относительно средних значений запасов
* для региона `1` эта среднеквадратичное отклонение предсказаний крайне мало, что говорит нам о том, что модель в регионе `1` достаточно надежно и точно предсказывает запасы сырья, относительно двух других регионов

На этапе расчета прибыли и рисков нам будет необходимо определить стоит ли рисковать, направляя работы в регионы с большими запасами (`0` и `2`), но менее предсказуемыми со стороны модели, или остановиться на регионе с меньшими запасами, но более стабильном по предсказаниям модели (`1`).

Для подсчета рисков мы внесли в проект все экономические показатели для расчета прибыли. Произведя нехитрые вычисления мы выяснили что:
* для безубыточной разработки месторождения в регионе необходимо, чтобы 200 наиболее перспективных по модели скважин в сумме имели запасы не ниже 22.2 млн баррелей
* в среднем запас сырья должен быть не ниже 111.1 тыс. баррелей на скважину  

Эти показатели превышают средние запасы сырья на скважину в регионах. То есть для безубыточной разработки необходимо, чтобы модель корректно выбирала топ-200 перспективных скважин в регионе.

Затем мы сделали **расчет прибыли** для самых перспективных скважин из всех, что имеются в регионе. На этом наиболее эффективными вновь оказались регионы `0` и `2`, регион `1` снова в аутсайдерах. Но стоит отметить, что все регионы при данном раскладе экономически эффективны.

Но такая ситуация практически исключена в реальной жизни. По условиям задания у нас есть 500 скважин в регионе для выбора 200 наиболее перспективных из них по модели. И мы с помощью техники bootstrap смоделировали 1000 случайных выборок в 500 скважин на каждом регионе и подсчитали средние экономические показатели эффективности для этих 1000 случаев.  

И при таком раскладе самым преспективным с точки зрения риска возникновения ситуации убыточного производства является регион `1`. В данном регионе только в 1.5% случаях производство будет убыточным (максимально возможная вероятность возникновения убытков по условиям задания - 2.5%). Также этот регион выделяется по наибольшей средней прибыли из всех перебранных 1000 вариантов - 452 млн рублей.  
    Регионы 0 и 2, несмотря на их больший максимальный потенциал по ресурсам, являются более рискованными и в 6 и 8% случаев соответственно месторождения в этих регионах окажутся убыточными.  
    
**Исходя из всего вышесказанного, для последующей разработки месторождения мы определенно будем рекомендовать руководству регион 1**